##### NB: This is Strictly for visualization of the data

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import geopandas as gpd
import plotly.express as px

In [ ]:
DATA_FRAME=pd.read_csv("../data/yield_df.csv")

DATA_FRAME.info()

### Inferences:
- Number of Entries: The dataset consists of 28242 entries, ranging from index 0 to 28241.
- Columns: There are 8 columns in the dataset.
> Data Types:
> - Most of the columns (6 out of 8) are of the int64 & float64 data type.
> - Only the Item and Area columns are of the object data type.
> - Missing Values: There doesn't appear to be any missing values in the dataset as each column has 28242 non-null entries.

In [ ]:
DATA_FRAME.describe().T

In [ ]:
DATA_FRAME.drop("Unnamed: 0", axis=1,inplace=True)
DATA_FRAME